In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
import sklearn.metrics

In [2]:
data = pd.read_excel("C:/Users/Donát/Documents/GitHub/-nkutfel/2019/2019_merged_matekA1A2_cleaned.xlsx")

In [3]:
#szűrjük ki a nem kategorikus vagy numerikus változókat
working_data = data.drop(labels = ["Name", "Neptun", "Tankör"], axis = 1)


#alakítsuk numerikussá a %-os teljesítménymutatókat
for i, row in working_data.iterrows():
    working_data.at[i, "Matek eredmény"] = float(row["Matek eredmény"][:-3] + "." + row["Matek eredmény"][-2])
    working_data.at[i, "Kognitív eredmény"] = float(row["Kognitív eredmény"][:-3] + "." + row["Kognitív eredmény"][-2])
    working_data.at[i, "Kognitív pont"] = float(str(row["Kognitív pont"]).replace(",", "."))
    working_data.at[i, "Össz eredmény (átlag)"] = float(row["Össz eredmény (átlag)"][:-3] + "." + row["Össz eredmény (átlag)"][-2])



**Innen jönnek a Vegyészek:**

In [4]:
#VEGYÉSZEKRE innen
#kiszedjük a matek helyes, összes pont és össz eredmény változókat mert redundánsak
from imblearn.over_sampling import SMOTE
vegyesz = working_data.query("Szak == 'Vegyészmérnöki'")
features = vegyesz.drop(labels = ["Matematika A1a - Analízis", "Matematika A2c", "Szak", "Matek helyes", "Összes pont", "Össz eredmény (átlag)"], axis = 1)
label = vegyesz["Matematika A1a - Analízis"]
#itt nem tudunk SMOTE-t alkalmazni, mert csak egy darab 1-es lett
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.3, random_state=0)

clf_veg = tree.DecisionTreeClassifier(random_state=0)
modell_veg = clf_veg.fit(X_train, y_train)

y_pred = modell_veg.predict(X_test)
y_tanito = modell_veg.predict(X_train)

print(f'Accuracy: {round(sklearn.metrics.accuracy_score(y_test, y_pred),2)}')
print(f'Precision: {round(sklearn.metrics.precision_score(y_test, y_pred, average = "macro"),2)}')
print(f'Recall: {round(sklearn.metrics.recall_score(y_test, y_pred, average = "macro"),2)}')


Accuracy: 0.22
Precision: 0.21
Recall: 0.16


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
#KNN-re keresztvalidációval mahalanobis-szal
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

print("With kNN:", end="\n")
neigh_dict_mahal = {}
for i in range(1,30):
    neigh = KNeighborsClassifier(n_neighbors=i, metric="mahalanobis", metric_params={'V': np.cov(m=X_train.astype(float), rowvar=False)})
    scores_kNN = cross_val_score(neigh, X_train.values, y_train.values, scoring = "accuracy", cv=10)
    neigh_dict_mahal[scores_kNN.mean()] = i
print("Optimal number of neighbors:", neigh_dict_mahal[max(list(neigh_dict_mahal.keys()))], end="\n")

With kNN:


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\ot

Optimal number of neighbors: 8


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


In [6]:
neigh = KNeighborsClassifier(n_neighbors=neigh_dict_mahal[max(list(neigh_dict_mahal.keys()))], metric="mahalanobis", metric_params={'V': np.cov(m=X_train.astype(float), rowvar=False)})
neigh = neigh.fit(X_train.values,y_train.values)
sk_pred = neigh.predict(X_test.to_numpy())
knn_mahal = round(sklearn.metrics.accuracy_score(y_test, sk_pred),2)
print(f'Accuracy: {round(sklearn.metrics.accuracy_score(y_test, sk_pred),2)}')


Accuracy: 0.42


In [7]:
#KNN-re keresztvalidációval euklidészi távolsággal
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

print("With kNN:", end="\n")
neigh_dict_eu = {}
for i in range(1,30):
    neigh = KNeighborsClassifier(n_neighbors=i, metric="euclidean")
    scores_kNN = cross_val_score(neigh, X_train.values, y_train.values, scoring = "accuracy", cv=10)
    neigh_dict_eu[scores_kNN.mean()] = i
print("Optimal number of neighbors:", neigh_dict_eu[max(list(neigh_dict_eu.keys()))], end="\n")

D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


With kNN:


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\ot

Optimal number of neighbors: 16


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


In [8]:
neigh = KNeighborsClassifier(n_neighbors=neigh_dict_eu[max(list(neigh_dict_eu.keys()))], metric="euclidean")
neigh = neigh.fit(X_train.values,y_train.values)
sk_pred = neigh.predict(X_test.to_numpy())
knn_eu = round(sklearn.metrics.accuracy_score(y_test, sk_pred),2)
print(f'Accuracy: {round(sklearn.metrics.accuracy_score(y_test, sk_pred),2)}')

Accuracy: 0.53


In [9]:
#összesítve kNN
print(f'kNN with Mahalanobis:')
print(f'- Optmial number of neighbours: {neigh_dict_mahal[max(list(neigh_dict_mahal.keys()))]} \n- Accuracy: {knn_mahal}')
print(f'kNN with Euclidean:')
print(f'- Optimal number of neighbours: {neigh_dict_eu[max(list(neigh_dict_eu.keys()))]} \n- Accuracy: {knn_eu}')

kNN with Mahalanobis:
- Optmial number of neighbours: 8 
- Accuracy: 0.42
kNN with Euclidean:
- Optimal number of neighbours: 16 
- Accuracy: 0.53


In [10]:
#tesztelés megfelelő C-hez SVM-nél
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

scores_svm = []
for i in range(1,100):
    clf = OneVsRestClassifier(SVC(C = i*0.1, class_weight = "balanced", random_state=0)).fit(X_train, y_train)    
    y_pred = clf.predict(X_test)
    scores_svm.append(round(sklearn.metrics.accuracy_score(y_test, y_pred),2))
print(scores_svm)

[0.44, 0.44, 0.53, 0.44, 0.56, 0.5, 0.56, 0.56, 0.56, 0.58, 0.53, 0.53, 0.53, 0.53, 0.53, 0.53, 0.53, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.53, 0.5, 0.5, 0.5, 0.5, 0.5, 0.53, 0.47, 0.5, 0.5, 0.47, 0.47, 0.47, 0.47, 0.47, 0.47, 0.44, 0.42, 0.42, 0.42, 0.42, 0.42, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.42, 0.42, 0.42, 0.42, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.47, 0.47, 0.47, 0.47, 0.47, 0.47, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44]


In [11]:
#keresztvalidációval
from sklearn.model_selection import cross_val_score

svm_dict = {}
for i in range(1,50):
    clf = OneVsRestClassifier(SVC(C = i*0.1, class_weight = "balanced", random_state=0))
    scores_svm = cross_val_score(clf, X_train, y_train, scoring = "accuracy", cv=10)
    svm_dict[scores_svm.mean()] = i/10
print("Optimal value for C:",svm_dict[max(list(svm_dict.keys()))])

D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\ot

D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\ot

Optimal value for C: 2.2


In [12]:
#SVM-re
clf = OneVsRestClassifier(SVC(C = svm_dict[max(list(svm_dict.keys()))], class_weight = "balanced", random_state=0)).fit(X_train, y_train)     # balanced nélkül romlik a pontosság
y_pred = clf.predict(X_test)
svm_acc = round(sklearn.metrics.accuracy_score(y_test, y_pred),2)

print(f'Accuracy: {round(sklearn.metrics.accuracy_score(y_test, y_pred),2)}')
print(f'Precision: {round(sklearn.metrics.precision_score(y_test, y_pred, average = "macro"),2)}')
print(f'Recall: {round(sklearn.metrics.recall_score(y_test, y_pred, average = "macro"),2)}')

Accuracy: 0.56
Precision: 0.36
Recall: 0.44


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
#logisztikus regresszióra
from sklearn.linear_model import LogisticRegression

logreg = OneVsRestClassifier(LogisticRegression(random_state=0, max_iter = 10000)).fit(X_train, y_train)
y_pred = logreg.predict(X_test)
log_acc = round(sklearn.metrics.accuracy_score(y_test, y_pred),2)

print(f'Accuracy: {round(sklearn.metrics.accuracy_score(y_test, y_pred),2)}')
print(f'Precision: {round(sklearn.metrics.precision_score(y_test, y_pred, average = "macro"),2)}')
print(f'Recall: {round(sklearn.metrics.recall_score(y_test, y_pred, average = "macro"),2)}')

Accuracy: 0.44
Precision: 0.34
Recall: 0.31


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
#Naive bayes-szel
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
bayes_acc = round(sklearn.metrics.accuracy_score(y_test, y_pred),2)

print(f'Accuracy: {round(sklearn.metrics.accuracy_score(y_test, y_pred),2)}')

Accuracy: 0.58


In [15]:
#előző 3 összegzése:
print("Accuracy with different algorithms:")
print(f'- SVM: {svm_acc} \n- Logistic Regression: {log_acc} \n- Naive Bayes: {bayes_acc}')

Accuracy with different algorithms:
- SVM: 0.56 
- Logistic Regression: 0.44 
- Naive Bayes: 0.58


In [16]:
#ezek után nem oneVSall: lineáris regresszió
from sklearn.linear_model import LinearRegression

reg_alap = LinearRegression().fit(features,label)
reg_alap.score(features, label)                        #nem túl magas

0.4629747599549636

In [17]:
#most nézzük a tanítóhalmazon és a teszthalmazon

reg = LinearRegression().fit(X_train,y_train)
reg_pred = reg.predict(X_test)

#együtthatók
print("Coefficients:", reg.coef_)
#RMSE

rmse = round(np.power(np.sum(np.power(y_test - reg_pred, 2))/reg_pred.shape[0],0.5),2)
print("RMSE:", rmse)

Coefficients: [ 9.28168409e-02  3.02961730e-01  4.47474385e+01  4.45501983e+01
  4.46821682e+01 -6.23425854e+00  6.42397818e+00 -4.23344261e+00
  2.12099913e-02  1.51810763e-02  4.73874796e-04  1.26934063e-02]
RMSE: 1.02


In [18]:
#R^2 és SSR
SST = np.sum(np.power(y_test-np.mean(y_test),2))
SSR = np.sum(np.power(reg_pred-np.mean(y_test),2))
R2 = round(SSR/SST,2)
R2

0.36

In [19]:
#kerekítsük a becsült értékeket a legközelebbi egészhez!
for i in range(reg_pred.shape[0]):
    reg_pred[i] = round(reg_pred[i],0)
    
rmse_rounded = round(np.power(np.sum(np.power(y_test - reg_pred, 2))/reg_pred.shape[0],0.5),2)
print("RMSE:", rmse_rounded)
lin_acc = round(sklearn.metrics.accuracy_score(y_test, reg_pred),2)
print(round(sklearn.metrics.accuracy_score(y_test, reg_pred),2))

RMSE: 1.05
0.22


In [20]:
#lineáris regresszió összesen:
print("Linear regression:")
print(f'- RMSE: {rmse} \n- R2: {R2} \n- RMSE after rounding: {rmse_rounded} \n- Accuracy after rounding: {lin_acc}')

Linear regression:
- RMSE: 1.02 
- R2: 0.36 
- RMSE after rounding: 1.05 
- Accuracy after rounding: 0.22


In [21]:
#összes teljes mindenes
#összesítve kNN
print(f'kNN with Mahalanobis:')
print(f'- Optmial number of neighbours: {neigh_dict_mahal[max(list(neigh_dict_mahal.keys()))]} \n- Accuracy: {knn_mahal}')
print(f'kNN with Euclidean:')
print(f'- Optimal number of neighbours: {neigh_dict_eu[max(list(neigh_dict_eu.keys()))]} \n- Accuracy: {knn_eu}')
print("SVM:")
print(f'- Accuracy: {svm_acc}')
print("Logistic Regression:")
print(f'- Accuracy: {log_acc}')
print("Naive Bayes:")
print(f'- Accuracy: {bayes_acc}')
print("Linear regression:")
print(f'- RMSE: {rmse} \n- R2: {R2} \n- RMSE after rounding: {rmse_rounded} \n- Accuracy after rounding: {lin_acc}')

kNN with Mahalanobis:
- Optmial number of neighbours: 8 
- Accuracy: 0.42
kNN with Euclidean:
- Optimal number of neighbours: 16 
- Accuracy: 0.53
SVM:
- Accuracy: 0.56
Logistic Regression:
- Accuracy: 0.44
Naive Bayes:
- Accuracy: 0.58
Linear regression:
- RMSE: 1.02 
- R2: 0.36 
- RMSE after rounding: 1.05 
- Accuracy after rounding: 0.22


In [22]:
#főkomponens analízissel
#nézzük, mennyi főkomponenssel lesz a legjobb. (pl a logisztikus regresszió)
from sklearn.decomposition import PCA

scores = []

for i in range(1,9):
    projected = PCA(i).fit_transform(features)
    X_train, X_test, y_train, y_test = train_test_split(projected, label, test_size=0.3, random_state=0)
    logreg = OneVsRestClassifier(LogisticRegression(random_state=0, max_iter = 10000)).fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    scores.append(round(sklearn.metrics.accuracy_score(y_test, y_pred),2))
    
scores

[0.47, 0.53, 0.42, 0.5, 0.47, 0.44, 0.39, 0.36]

In [23]:
#főkomponenses logreg
from sklearn.decomposition import PCA
projected = PCA(3).fit_transform(features)
X_train, X_test, y_train, y_test = train_test_split(projected, label, test_size=0.3, random_state=0)

logreg = OneVsRestClassifier(LogisticRegression(random_state=0, max_iter = 10000)).fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print(f'Accuracy: {round(sklearn.metrics.accuracy_score(y_test, y_pred),2)}')
print(f'Precision: {round(sklearn.metrics.precision_score(y_test, y_pred, average = "macro"),2)}')
print(f'Recall: {round(sklearn.metrics.recall_score(y_test, y_pred, average = "macro"),2)}')


Accuracy: 0.42
Precision: 0.32
Recall: 0.34


In [24]:
#nézzük meg, hogy főkompononensenként hogy változnak a különböző algoritmusok eredménymutatói, és rakjuk dataframe-be!
PCA_scores_vegy = pd.DataFrame({"Főkomponensek száma":[], "Logreg Acc.":[], "SVM Acc.":[], "Linreg RMSE":[], "Linreg R2":[], 
                           "Linreg RMSE kerekített":[], "Linreg Acc kerekített":[]})


for i in range(1,9):
    projected = PCA(i).fit_transform(features)
    X_train, X_test, y_train, y_test = train_test_split(projected, label, test_size=0.3, random_state=0)
    
    logreg = OneVsRestClassifier(LogisticRegression(random_state=0, max_iter = 10000)).fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    log_acc = round(sklearn.metrics.accuracy_score(y_test, y_pred),2)
    
    clf = OneVsRestClassifier(SVC(class_weight = "balanced", random_state=0)).fit(X_train, y_train)    
    y_pred = clf.predict(X_test)
    SVM_acc = round(sklearn.metrics.accuracy_score(y_test, y_pred),2)
    
    reg = LinearRegression().fit(X_train,y_train)
    reg_pred = reg.predict(X_test)
    rmse = round(np.power(np.sum(np.power(y_test - reg_pred, 2))/reg_pred.shape[0],0.5),2)
    SST = np.sum(np.power(y_test-np.mean(y_test),2))
    SSR = np.sum(np.power(reg_pred-np.mean(y_test),2))
    R2 = round(SSR/SST,2)
    
    for j in range(reg_pred.shape[0]):
        reg_pred[j] = round(reg_pred[j],0)
    rmse_rounded = round(np.power(np.sum(np.power(y_test - reg_pred, 2))/reg_pred.shape[0],0.5),2)
    reg_pred_rounded = round(sklearn.metrics.accuracy_score(y_test, reg_pred),2)
    
    #új sor a dataframe-be
    new_row = {"Főkomponensek száma":i, "Logreg Acc.":log_acc, "SVM Acc.":SVM_acc, "Linreg RMSE":rmse, "Linreg R2":R2, 
               "Linreg RMSE kerekített":rmse_rounded, "Linreg Acc kerekített":reg_pred_rounded}
    PCA_scores_vegy = PCA_scores_vegy.append(new_row, ignore_index=True)

In [25]:
PCA_scores_vegy

,Főkomponensek száma,Logreg Acc.,SVM Acc.,Linreg RMSE,Linreg R2,Linreg RMSE kerekített,Linreg Acc kerekített
0,1.0,0.47,0.47,0.90,0.29,0.94,0.28
1,2.0,0.53,0.44,0.92,0.29,0.94,0.28
2,3.0,0.42,0.42,0.88,0.31,0.90,0.28
3,4.0,0.50,0.42,0.90,0.31,0.90,0.28
4,5.0,0.47,0.36,0.91,0.30,0.96,0.33
5,6.0,0.44,0.36,0.90,0.30,0.97,0.31
6,7.0,0.39,0.36,0.90,0.29,0.96,0.33
7,8.0,0.36,0.36,0.92,0.31,1.05,0.22


**Biomérnököknél:**

In [26]:
#BIOMÉRNÖKÖKRE innen

bio = working_data.query("Szak == 'Biomérnöki'")
features = bio.drop(labels = ["Matematika A1a - Analízis", "Matematika A2c", "Szak"], axis = 1)
label = bio["Matematika A1a - Analízis"]
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.3, random_state=0)

clf_veg = tree.DecisionTreeClassifier(random_state=0)
modell_veg = clf_veg.fit(X_train, y_train)

y_pred = modell_veg.predict(X_test)
y_tanito = modell_veg.predict(X_train)

print(f'Accuracy: {round(sklearn.metrics.accuracy_score(y_test, y_pred),2)}')
print(f'Precision: {round(sklearn.metrics.precision_score(y_test, y_pred, average = "macro"),2)}')
print(f'Recall: {round(sklearn.metrics.recall_score(y_test, y_pred, average = "macro"),2)}')


Accuracy: 0.24
Precision: 0.21
Recall: 0.22


In [27]:
#KNN-re keresztvalidációval mahalanobis-szal
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

print("With kNN:", end="\n")
neigh_dict_mahal = {}
for i in range(1,30):
    neigh = KNeighborsClassifier(n_neighbors=i, metric="mahalanobis", metric_params={'V': np.cov(m=X_train.astype(float), rowvar=False)})
    scores_kNN = cross_val_score(neigh, X_train.values, y_train.values, scoring = "accuracy", cv=10)
    neigh_dict_mahal[scores_kNN.mean()] = i
print("Optimal number of neighbors:", neigh_dict_mahal[max(list(neigh_dict_mahal.keys()))], end="\n")

With kNN:


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args,

D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otth

D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args,

D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args,

Optimal number of neighbors: 20


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args,

In [28]:
neigh = KNeighborsClassifier(n_neighbors=neigh_dict_mahal[max(list(neigh_dict_mahal.keys()))], metric="mahalanobis", metric_params={'V': np.cov(m=X_train.astype(float), rowvar=False)})
neigh = neigh.fit(X_train.values,y_train.values)
sk_pred = neigh.predict(X_test.to_numpy())
knn_mahal = round(sklearn.metrics.accuracy_score(y_test, sk_pred),2)
print(f'Accuracy: {round(sklearn.metrics.accuracy_score(y_test, sk_pred),2)}')


Accuracy: 0.28


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\neighbors\_base.py:600: RuntimeWarning: invalid value encountered in sqrt
  return tree.query(*args, **kwargs)


In [29]:
#KNN-re keresztvalidációval euklidészi távolsággal
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

print("With kNN:", end="\n")
neigh_dict_eu = {}
for i in range(1,30):
    neigh = KNeighborsClassifier(n_neighbors=i, metric="euclidean")
    scores_kNN = cross_val_score(neigh, X_train.values, y_train.values, scoring = "accuracy", cv=10)
    neigh_dict_eu[scores_kNN.mean()] = i
print("Optimal number of neighbors:", neigh_dict_eu[max(list(neigh_dict_eu.keys()))], end="\n")

With kNN:


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\ot

Optimal number of neighbors: 18


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(


In [30]:
neigh = KNeighborsClassifier(n_neighbors=neigh_dict_eu[max(list(neigh_dict_eu.keys()))], metric="euclidean")
neigh = neigh.fit(X_train.values,y_train.values)
sk_pred = neigh.predict(X_test.to_numpy())
knn_eu = round(sklearn.metrics.accuracy_score(y_test, sk_pred),2)
print(f'Accuracy: {round(sklearn.metrics.accuracy_score(y_test, sk_pred),2)}')

Accuracy: 0.36


In [31]:
#összesítve kNN
print(f'kNN with Mahalanobis:')
print(f'- Optmial number of neighbours: {neigh_dict_mahal[max(list(neigh_dict_mahal.keys()))]} \n- Accuracy: {knn_mahal}')
print(f'kNN with Euclidean:')
print(f'- Optimal number of neighbours: {neigh_dict_eu[max(list(neigh_dict_eu.keys()))]} \n- Accuracy: {knn_eu}')

kNN with Mahalanobis:
- Optmial number of neighbours: 20 
- Accuracy: 0.28
kNN with Euclidean:
- Optimal number of neighbours: 18 
- Accuracy: 0.36


In [32]:
#tesztelés megfelelő C-hez SVM-nél
scores_svm = []
for i in range(1,100):
    clf = OneVsRestClassifier(SVC(C = i*0.1, class_weight = "balanced", random_state=0)).fit(X_train, y_train)    
    y_pred = clf.predict(X_test)
    scores_svm.append(round(sklearn.metrics.accuracy_score(y_test, y_pred),2))
print(scores_svm)

[0.04, 0.04, 0.12, 0.04, 0.16, 0.12, 0.12, 0.04, 0.4, 0.16, 0.08, 0.12, 0.32, 0.12, 0.12, 0.04, 0.28, 0.4, 0.08, 0.2, 0.0, 0.08, 0.28, 0.12, 0.44, 0.2, 0.04, 0.16, 0.16, 0.16, 0.16, 0.0, 0.12, 0.16, 0.16, 0.2, 0.16, 0.12, 0.12, 0.12, 0.08, 0.16, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.16, 0.16, 0.16, 0.16, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16]


In [33]:
from sklearn.model_selection import cross_val_score

svm_dict = {}
for i in range(1,50):
    clf = OneVsRestClassifier(SVC(C = i*0.1, class_weight = "balanced", random_state=0))
    scores_svm = cross_val_score(clf, X_train, y_train, scoring = "accuracy", cv=10)
    svm_dict[scores_svm.mean()] = i/10
print("Optimal value for C:",svm_dict[max(list(svm_dict.keys()))])

D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\ot

D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
D:\anaconda3\envs\ot

Optimal value for C: 1.3


In [34]:
#SVM-re
clf = OneVsRestClassifier(SVC(C = svm_dict[max(list(svm_dict.keys()))], class_weight = "balanced", random_state=0)).fit(X_train, y_train)     # balanced nélkül romlik a pontosság
y_pred = clf.predict(X_test)
svm_acc = round(sklearn.metrics.accuracy_score(y_test, y_pred),2)

print(f'Accuracy: {round(sklearn.metrics.accuracy_score(y_test, y_pred),2)}')
print(f'Precision: {round(sklearn.metrics.precision_score(y_test, y_pred, average = "macro"),2)}')
print(f'Recall: {round(sklearn.metrics.recall_score(y_test, y_pred, average = "macro"),2)}')

Accuracy: 0.32
Precision: 0.39
Recall: 0.32


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
#logisztikus regresszióra

logreg = OneVsRestClassifier(LogisticRegression(random_state=0, max_iter = 10000)).fit(X_train, y_train)
y_pred = logreg.predict(X_test)
log_acc = round(sklearn.metrics.accuracy_score(y_test, y_pred),2)

print(f'Accuracy: {round(sklearn.metrics.accuracy_score(y_test, y_pred),2)}')
print(f'Precision: {round(sklearn.metrics.precision_score(y_test, y_pred, average = "macro"),2)}')
print(f'Recall: {round(sklearn.metrics.recall_score(y_test, y_pred, average = "macro"),2)}')

Accuracy: 0.12
Precision: 0.06
Recall: 0.08


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
#Naive bayes-szel
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
bayes_acc = round(sklearn.metrics.accuracy_score(y_test, y_pred),2)

print(f'Accuracy: {round(sklearn.metrics.accuracy_score(y_test, y_pred),2)}')

Accuracy: 0.32


In [37]:
#az előző 3 kiértékelése
print("Accuracy with different algorithms:")
print(f'- SVM: {svm_acc} \n- Logistic Regression: {log_acc} \n- Naive Bayes: {bayes_acc}')

Accuracy with different algorithms:
- SVM: 0.32 
- Logistic Regression: 0.12 
- Naive Bayes: 0.32


In [38]:
#lineáris regresszió

reg = LinearRegression().fit(X_train,y_train)
reg_pred = reg.predict(X_test)

#együtthatók
print("Coefficients:", reg.coef_)
#RMSE

rmse = round(np.power(np.sum(np.power(y_test - reg_pred, 2))/reg_pred.shape[0],0.5),2)
print("RMSE:", rmse)

Coefficients: [-2.77581180e-01  1.28156837e+00 -2.28816496e+09 -2.28816496e+09
 -2.28816496e+09  2.28816501e+09 -6.98190884e+00 -8.79111540e+00
  4.83546799e+00  1.88259145e+00  6.04660346e-04  2.12817026e+11
  2.12817026e+11  2.12817026e+11 -2.12817026e+11]
RMSE: 1.08


In [39]:
#R^2 és SSR
SST = np.sum(np.power(y_test-np.mean(y_test),2))
SSR = np.sum(np.power(reg_pred-np.mean(y_test),2))
R2 = round(SSR/SST,2)
R2

0.32

In [40]:
#kerekítsük a becsült értékeket a legközelebbi egészhez!
for i in range(reg_pred.shape[0]):
    reg_pred[i] = round(reg_pred[i],0)
    
rmse_rounded = round(np.power(np.sum(np.power(y_test - reg_pred, 2))/reg_pred.shape[0],0.5),2)
print("RMSE:", rmse_rounded)
lin_acc = round(sklearn.metrics.accuracy_score(y_test, reg_pred),2)
print(round(sklearn.metrics.accuracy_score(y_test, reg_pred),2))

RMSE: 1.13
0.32


In [41]:
#lineáris regresszió összesen:
print("Linear regression:")
print(f'- RMSE: {rmse} \n- R2: {R2} \n- RMSE after rounding: {rmse_rounded} \n- Accuracy after rounding: {lin_acc}')

Linear regression:
- RMSE: 1.08 
- R2: 0.32 
- RMSE after rounding: 1.13 
- Accuracy after rounding: 0.32


In [42]:
#összes teljes mindenes
#összesítve kNN
print(f'kNN with Mahalanobis:')
print(f'- Optmial number of neighbours: {neigh_dict_mahal[max(list(neigh_dict_mahal.keys()))]} \n- Accuracy: {knn_mahal}')
print(f'kNN with Euclidean:')
print(f'- Optimal number of neighbours: {neigh_dict_eu[max(list(neigh_dict_eu.keys()))]} \n- Accuracy: {knn_eu}')
print("SVM:")
print(f'- Accuracy: {svm_acc}')
print("Logistic Regression:")
print(f'- Accuracy: {log_acc}')
print("Naive Bayes:")
print(f'- Accuracy: {bayes_acc}')
print("Linear regression:")
print(f'- RMSE: {rmse} \n- R2: {R2} \n- RMSE after rounding: {rmse_rounded} \n- Accuracy after rounding: {lin_acc}')

kNN with Mahalanobis:
- Optmial number of neighbours: 20 
- Accuracy: 0.28
kNN with Euclidean:
- Optimal number of neighbours: 18 
- Accuracy: 0.36
SVM:
- Accuracy: 0.32
Logistic Regression:
- Accuracy: 0.12
Naive Bayes:
- Accuracy: 0.32
Linear regression:
- RMSE: 1.08 
- R2: 0.32 
- RMSE after rounding: 1.13 
- Accuracy after rounding: 0.32


In [43]:
#főkomponens analízissel
#nézzük, mennyi főkomponenssel lesz a legjobb. (pl a logisztikus regresszió)
from sklearn.decomposition import PCA

scores = []

for i in range(1,9):
    projected = PCA(i).fit_transform(features)
    X_train, X_test, y_train, y_test = train_test_split(projected, label, test_size=0.3, random_state=0)
    logreg = OneVsRestClassifier(LogisticRegression(random_state=0, max_iter = 10000)).fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    scores.append(round(sklearn.metrics.accuracy_score(y_test, y_pred),2))
    
scores

[0.32, 0.4, 0.4, 0.4, 0.4, 0.4, 0.36, 0.32]

In [44]:
#főkomponenses logreg
from sklearn.decomposition import PCA
projected = PCA(3).fit_transform(features)
X_train, X_test, y_train, y_test = train_test_split(projected, label, test_size=0.3, random_state=0)

logreg = OneVsRestClassifier(LogisticRegression(random_state=0, max_iter = 10000)).fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print(f'Accuracy: {round(sklearn.metrics.accuracy_score(y_test, y_pred),2)}')
print(f'Precision: {round(sklearn.metrics.precision_score(y_test, y_pred, average = "macro"),2)}')
print(f'Recall: {round(sklearn.metrics.recall_score(y_test, y_pred, average = "macro"),2)}')

#nagyon jó eredmény

Accuracy: 0.4
Precision: 0.2
Recall: 0.27


D:\anaconda3\envs\otthoni\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
#nézzük meg, hogy főkompononensenként hogy változnak a különböző algoritmusok eredménymutatói, és rakjuk dataframe-be!
PCA_scores_bio = pd.DataFrame({"Főkomponensek száma":[], "Logreg Acc.":[], "SVM Acc.":[], "Linreg RMSE":[], "Linreg R2":[], 
                           "Linreg RMSE kerekített":[], "Linreg Acc kerekített":[]})


for i in range(1,9):
    projected = PCA(i).fit_transform(features)
    X_train, X_test, y_train, y_test = train_test_split(projected, label, test_size=0.3, random_state=0)
    
    logreg = OneVsRestClassifier(LogisticRegression(random_state=0, max_iter = 10000)).fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    log_acc = round(sklearn.metrics.accuracy_score(y_test, y_pred),2)
    
    clf = OneVsRestClassifier(SVC(class_weight = "balanced", random_state=0)).fit(X_train, y_train)    
    y_pred = clf.predict(X_test)
    SVM_acc = round(sklearn.metrics.accuracy_score(y_test, y_pred),2)
    
    reg = LinearRegression().fit(X_train,y_train)
    reg_pred = reg.predict(X_test)
    rmse = round(np.power(np.sum(np.power(y_test - reg_pred, 2))/reg_pred.shape[0],0.5),2)
    SST = np.sum(np.power(y_test-np.mean(y_test),2))
    SSR = np.sum(np.power(reg_pred-np.mean(y_test),2))
    R2 = round(SSR/SST,2)
    
    for j in range(reg_pred.shape[0]):
        reg_pred[j] = round(reg_pred[j],0)
    rmse_rounded = round(np.power(np.sum(np.power(y_test - reg_pred, 2))/reg_pred.shape[0],0.5),2)
    reg_pred_rounded = round(sklearn.metrics.accuracy_score(y_test, reg_pred),2)
    
    #új sor a dataframe-be
    new_row = {"Főkomponensek száma":i, "Logreg Acc.":log_acc, "SVM Acc.":SVM_acc, "Linreg RMSE":rmse, "Linreg R2":R2, 
               "Linreg RMSE kerekített":rmse_rounded, "Linreg Acc kerekített":reg_pred_rounded}
    PCA_scores_bio = PCA_scores_bio.append(new_row, ignore_index=True)

In [46]:
PCA_scores_bio

,Főkomponensek száma,Logreg Acc.,SVM Acc.,Linreg RMSE,Linreg R2,Linreg RMSE kerekített,Linreg Acc kerekített
0,1.0,0.32,0.16,0.98,0.05,0.98,0.40
1,2.0,0.40,0.32,0.96,0.12,1.02,0.40
2,3.0,0.40,0.40,0.95,0.23,1.04,0.24
3,4.0,0.40,0.28,0.96,0.27,1.02,0.28
4,5.0,0.40,0.32,0.94,0.22,1.00,0.32
5,6.0,0.40,0.28,0.96,0.17,1.08,0.28
6,7.0,0.36,0.28,0.97,0.18,1.06,0.32
7,8.0,0.32,0.28,1.03,0.21,1.02,0.28
